In [1]:
"""Code to format info on movies in a csv file for publication."""

# built-in libraries
import csv
import datetime
from email.message import EmailMessage
import os.path
import re
import smtplib
import sqlite3
import sys

# installed with pip
import pandas as pd
#from sqlalchemy import create_engine



In [2]:
# Create list of dictionaries for each row.
listings = []
scrape_file = 'newscrape.csv'
with open(scrape_file, 'r') as f:
    reader = csv.reader(f)
    for row in reader:
        layout_dct = {
            'addy': row[0],
            'theater': row[1],
            'title': row[2],
            'mpaa': row[3],
            'stars': row[4],
            'times': row[5],
            'format': row[6],
            'synopsis': row[7],
            'date': row[8],
            'city': row[9]
        }
        listings.append(layout_dct)
        
del listings[0] #remove header row   

In [5]:
# Create Pandas dataframe for listings
df = pd.DataFrame(listings)
# Remove some duplicate listings
df = df[df.city != "Belleview, FL"]
df = df[df.city != "High Springs, FL"]
df = df[(df.city != "The Villages, FL") & (df.theater != "Belleview Cinemas")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
df = df[(df.city != "The Villages, FL") & (df.theater != "Old Mill Playhouse")]
times = (df.times.str.split('|', expand=True).rename(columns=lambda x: f"time_{x+1}"))
times = times.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('p', ' p.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace('a', ' a.m.') if x.dtype == "object" else x)
times = times.apply(lambda x: x.str.replace(':00', '') if x.dtype == "object" else x)
df = df.drop('times', axis=1)
df = pd.concat([df, times], axis=1, join_axes=[df.index])
df['streetaddy'] = df.addy.str.split(',').str[0]
df.streetaddy = df.streetaddy.str.replace('Blvd', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Blvd..', 'Blvd.')
df.streetaddy = df.streetaddy.str.replace('Avenue', 'Ave')
df.streetaddy = df.streetaddy.str.replace('Ave', 'Ave.')
df.streetaddy = df.streetaddy.str.replace('Ave..', 'Ave.')
df['date'] = pd.to_datetime(df['date'], errors='coerce')
df['movieday'] = df['date'].dt.weekday_name
df = df.drop('city', axis=1)
df['cityaddy'] = df.addy.str.split(',').str[1]
df = df.apply(lambda x: x.str.strip() if x.dtype == "object" else x)
df = df.drop('addy', axis=1)
df.head()

,date,format,mpaa,stars,synopsis,theater,title,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,streetaddy,movieday,cityaddy
0,2019-01-23,Standard,PG,5,"Spider-Man mentors a teenager from Brooklyn, N...",Celebration Pointe 10,Spider-Man: Into the Spider-Verse,4:20 p.m.,7:20 p.m.,10:40 p.m.,None,None,None,None,None,4901 SW 31st Place,Wednesday,Gainesville
7,2019-01-23,Standard,PG-13,4,Security guard David Dunn uses his supernatura...,Regal Royal Park Stadium 16,Glass (2019),3:50 p.m.,7 p.m.,9:55 p.m.,None,None,None,None,None,3702 Newberry Road,Wednesday,Gainesville
8,2019-01-23,Standard,PG-13,4.5,Phillip is a wealthy quadriplegic who needs a ...,UA Cinema 90 6,The Upside,3:45 p.m.,7:10 p.m.,None,None,None,None,None,None,2830 Hwy 90 West,Wednesday,Lake City
10,2019-01-23,IMAX,PG-13,4,Security guard David Dunn uses his supernatura...,Regal Hollywood Stadium 16 & IMAX - Ocala,Glass (2019),3 p.m.,7 p.m.,10 p.m.,None,None,None,None,None,2801 SW 27th Ave.,Wednesday,Ocala
14,2019-01-23,Standard,R,4,"Broke, alone and facing foreclosure on his bus...",Regal Butler Town Center 14,The Mule (2018),3:40 p.m.,10:30 p.m.,None,None,None,None,None,None,3101 SW 35th Blvd.,Wednesday,Gainesville


In [ ]:
# Put new cinema scrape data into db
#db_directory = os.path.dirname(os.path.abspath(__file__))
#db_file = os.path.join(db_directory, "rinspect.sqlite")

db_file = "cinema_db.sqlite"
movie_table = 'movies' # table for movie data
conn = sqlite3.connect(db_file)
cur = conn.cursuor()

cur.execute("insert into airlines values ()




In [ ]:
def df2sqlite(df, db_name = "cinema_db.sqlite", tbl_name = "movies"):
 
  conn=sqlite3.connect(db_name)
  cur = conn.cursor()                                 
 
  wildcards = ','.join(['?'] * len(df.columns))              
  data = [tuple(x) for x in df.values]
 
  cur.execute("drop table if exists %s" % tbl_name)
 
  col_str = '"' + '","'.join(df.columns) + '"'
  cur.execute("create table %s (%s)" % (tbl_name, col_str))
 
  cur.executemany("insert into %s values(%s)" % (tbl_name, wildcards), data)
 
  conn.commit()
  conn.close()

In [ ]:
df2sqlite(df)

In [6]:
# Ocala market
oc6_df = df.loc[df['theater'] == 'Ocala Center 6']
ocd_dr = df.loc[df['theater'] == 'Ocala Drive-In']
bv_df = df.loc[df['theater'] == 'Belleview Cinemas']
mt_df = df.loc[df['theater'] == 'Marion Theatre']
omp_df = df.loc[df['theater'] == 'Old Mill Playhouse']
regaloca_df = df.loc[df['theater'] == 'Regal Hollywood Stadium 16 & IMAX - Ocala']
rialto_df = df.loc[df['theater'] == 'Rialto Theatre Spanish Springs Town Square']
amc_df = df.loc[df['theater'] == 'AMC Lake Square 12']
barn_df = df.loc[df['theater'] == 'Barnstorm Theater']

# Gainesville market
celeb_df = df.loc[df['theater'] == 'Celebration Pointe 10']
hipp_df = df.loc[df['theater'] == 'Hippodrome State Theatre']
butler_df = df.loc[df['theater'] == 'Regal Butler Town Center 14']
regalgnv_df = df.loc[df['theater'] == 'Regal Royal Park Stadium 16']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']
starke_df = df.loc[df['theater'] == 'Florida Twin']
uac_df = df.loc[df['theater'] == 'UA Cinema 90 6']

In [7]:
oc6_df.head()
    

,date,format,mpaa,stars,synopsis,theater,title,time_1,time_2,time_3,time_4,time_5,time_6,time_7,time_8,streetaddy,movieday,cityaddy
51,2019-01-23,Standard,PG-13,4,Security guard David Dunn uses his supernatura...,Ocala Center 6,Glass (2019),1 p.m.,4 p.m.,7 p.m.,7:15 p.m.,None,None,None,None,2021 Silver Springs Blvd.,Wednesday,Ocala
64,2019-01-23,Standard,PG-13,3.5,William Foster is a brilliant neuroscientist w...,Ocala Center 6,Replicas,1 p.m.,4 p.m.,None,None,None,None,None,None,2021 Silver Springs Blvd.,Wednesday,Ocala
126,2019-01-23,Standard,PG-13,4.5,"On the run in the year 1987, Bumblebee the Aut...",Ocala Center 6,Bumblebee,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Wednesday,Ocala
127,2019-01-23,Standard,PG,4.5,"As a puppy, Bella finds her way into the arms ...",Ocala Center 6,A Dog's Way Home,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Wednesday,Ocala
177,2019-01-23,Standard,PG-13,4.5,Aquaman finds himself caught between a surface...,Ocala Center 6,Aquaman,1 p.m.,4 p.m.,7 p.m.,None,None,None,None,None,2021 Silver Springs Blvd.,Wednesday,Ocala


In [ ]:
# Convert datetime object into day of the week; But this is accomplished in the df work already.
def get_day(date_object=None):
    if not date_object:
        date_object = datetime.datetime.now()
    stamp = ""
    stamp += datetime.datetime.strftime(date_object, "%A")
    return(stamp)